In [1]:
# Relations and duplicated data between datasets

This document describes relations between datasets and some inconsistencies that were found.

The following are the two main column identifier keys across datasets:
- OBLIGACION: identifier that links CARTERA, CASTIGO and COLOCACION datasets.
- CLIENTE (former "Homologacion Documento de Identidad"): identifier that links CARTERA, CONTACTO, and NEGOCIO datasets.

## Setting up notebook

In [2]:
import json
import os

import pandas as pd
import numpy as np

from dotenv import load_dotenv

from core_ds4a_project import cleaning
from core_ds4a_project import columns as project_columns
from core_ds4a_project import datasets

%load_ext autoreload
%autoreload 1
%aimport core_ds4a_project, core_ds4a_project.cleaning, core_ds4a_project.columns, core_ds4a_project.datasets

In [3]:
pd.set_option("display.max_columns", 60)

Environment variables:

In [4]:
load_dotenv('envvars')

ROOT_DATA_PATH = os.environ.get('ROOT_DATA_PATH')
RAW_DATA_PATH = os.environ.get('RAW_DATA_PATH') or f'{ROOT_DATA_PATH}/raw'

Reading data:

In [5]:
dir_path = RAW_DATA_PATH
cartera_df = datasets.read_cartera(dir_path, clean_all=True)
castigo_df = datasets.read_castigada_xlsx(dir_path, clean=True)
colocacion_df = datasets.read_colocacion_xlsx(dir_path, clean=True)
contacto_df = datasets.read_contacto(dir_path, clean=True)
negocio_df = datasets.read_negocio(dir_path, clean=True)

In [6]:
cartera_df.shape, castigo_df.shape, colocacion_df.shape, contacto_df.shape, negocio_df.shape

((790234, 41), (1170, 15), (42009, 33), (60043, 24), (37375, 147))

Functions:

In [7]:
def count_indices(eq, diff, na_any, na_both, na_single):
    return {
        "total_rows": df.shape[0],
        "count_eq": eq.sum(),
        "count_diff": diff.sum(),
        "count_na_any": na_any.sum(),
        "count_na_single": na_single.sum(),
        "count_na_both": na_both.sum(),
    }


## Relations: CARTERA, COLOCACION, CONTACTO, NEGOCIO

There are some inconsistencies in the relations between CARTERA and COLOCACION datasets for OBLIGACION identifier:
- There are different number of unique OBLIGACION identifiers in each dataset.
- There are OBLIGACION identifiers in CARTERA that are not present in COLOCACION and viceversa, there are OBLIGACION identifiers in COLOCACION that are not present in CARTERA.

In [8]:
cartera_obligacion_ids = cartera_df['OBLIGACION'].dropna().drop_duplicates()
colocacion_obligacion_ids = colocacion_df['OBLIGACION'].dropna().drop_duplicates()

index_cartera_obligacion_in_colocacion = cartera_obligacion_ids.isin(colocacion_obligacion_ids)
index_colocacion_obligacion_in_cartera = colocacion_obligacion_ids.isin(cartera_obligacion_ids)

{
    "count_cartera_obligacion_ids": len(cartera_obligacion_ids),
    "count_colocacion_obligacion_ids": len(colocacion_obligacion_ids),
    "count_cartera_obligacion_ids_not_in_colocacion": (~index_cartera_obligacion_in_colocacion).sum(),
    "count_colocacion_obligacion_ids_not_in_cartera": (~index_colocacion_obligacion_in_cartera).sum()
}


{'count_cartera_obligacion_ids': 56533,
 'count_colocacion_obligacion_ids': 42009,
 'count_cartera_obligacion_ids_not_in_colocacion': 15291,
 'count_colocacion_obligacion_ids_not_in_cartera': 767}

We're then considering valid only those records whose OBLIGACION identifier is present in both CARTERA and COLOCACION, validating there's a single CLIENTE for each OBLIGACION:

In [9]:
cliente_obligacion_df = (
    cartera_df
    [['CLIENTE', 'OBLIGACION']]
    .dropna()
    .drop_duplicates()
    .query('OBLIGACION.isin(@colocacion_obligacion_ids)')
)

counts_clientes_per_obligacion = cliente_obligacion_df.groupby('OBLIGACION').count()['CLIENTE']

{
    "count_obligaciones": len(cliente_obligacion_df),
    "each_obligacion_has_single_cliente": (counts_clientes_per_obligacion == 1).all(),
}

{'count_obligaciones': 41242, 'each_obligacion_has_single_cliente': True}

All valid records in CARTERA are found in CONTACTO and NEGOCIO:

In [10]:
cliente_ids = cliente_obligacion_df['CLIENTE'].drop_duplicates().reset_index(drop=True)

index_cliente_in_concacto = cliente_ids.isin(contacto_df['CLIENTE'])
index_cliente_in_negocio = cliente_ids.isin(negocio_df['CLIENTE'])
index_clientes_in_contacto_n_negocio = index_cliente_in_concacto & index_cliente_in_negocio

valid_cliente_ids = cliente_ids[index_clientes_in_contacto_n_negocio]
cliente_obligacion_df = cliente_obligacion_df.query('CLIENTE.isin(@valid_cliente_ids)')
valid_obligacion_ids = cliente_obligacion_df['OBLIGACION'].drop_duplicates()

{
    "count_clientes": len(cliente_ids),
    "count_clientes_in_contacto": index_cliente_in_concacto.sum(),
    "count_clientes_in_negocio": index_cliente_in_negocio.sum(),
    "count_clientes_in_contacto_n_negocio": index_clientes_in_contacto_n_negocio.sum()
}

{'count_clientes': 24127,
 'count_clientes_in_contacto': 24127,
 'count_clientes_in_negocio': 22965,
 'count_clientes_in_contacto_n_negocio': 22965}

Validating function in local package:

In [11]:
(cids, oids) = datasets.get_valid_relations_ids(cartera_df, colocacion_df, contacto_df, negocio_df)

valid_cliente_ids.equals(cids), valid_obligacion_ids.equals(oids)

(True, True)

In [12]:
kws = {
    "valid_cliente_ids": list(valid_cliente_ids),
    "valid_obligacion_ids": list(valid_obligacion_ids.astype(int))
}

datasets.RelationsFilter(**kws).save(ROOT_DATA_PATH)

## Filtering: CARTERA

Considering some clients in CARTERA have credits that are both known and unknown in COLOCACION:

In [13]:
df = (
    cartera_df[['CLIENTE', 'OBLIGACION']]
    .dropna()
    .drop_duplicates()
    .query('CLIENTE.isin(@valid_cliente_ids)')
    )
count_all_obligacion_ids_per_cliente_id = (
    df
    .groupby('CLIENTE')
    .count()
    .rename(columns={'OBLIGACION': 'COUNT_ALL_OBLIGACION'})
)
count_unknown_obligacion_ids_per_cliente_id = (
    df
    .query('~OBLIGACION.isin(@valid_obligacion_ids)')
    .groupby('CLIENTE')
    .count()
    .rename(columns={'OBLIGACION': 'COUNT_UNKNOWN_OBLIGACION'})
)

counts_cliente_obligacion_df = (
    df
    [['CLIENTE']]
    .drop_duplicates()
    .merge(count_all_obligacion_ids_per_cliente_id, on='CLIENTE')
    .merge(count_unknown_obligacion_ids_per_cliente_id, on='CLIENTE')
)

counts_cliente_obligacion_df

,CLIENTE,COUNT_ALL_OBLIGACION,COUNT_UNKNOWN_OBLIGACION
0,FA24149,2,1
1,FA15645,3,2
2,FA23817,2,1
3,FA22202,2,1
4,FA6902,2,1
...,...,...,...
5747,FA22939,2,1
5748,FA2656,2,1
5749,FA18621,3,1
5750,FA18537,2,1


CARTERA must be filtered by both valid CLIENTE identifiers and valid OBLIGACION identifiers:

In [14]:
cartera_df.shape

(790234, 41)

In [15]:
cartera_df = (
    cartera_df
    .query('CLIENTE.isin(@valid_cliente_ids) & OBLIGACION.isin(@valid_obligacion_ids)')
)
cartera_df.shape

(569129, 41)

## Filtering: CASTIGO

There are some inconsistencies in the relations between CASTIGO and other two financial CARTERA and COLOCACION datasets for OBLIGACION identifier:

In [16]:
{
    "count_castigo_obligacion": castigo_df.shape[0],
    "count_castigo_obligacion_in_cartera": castigo_df['OBLIGACION'].isin(cartera_obligacion_ids).sum(),
    "count_castigo_obligacion_in_colocacion": castigo_df['OBLIGACION'].isin(colocacion_obligacion_ids).sum(),
    "count_castigo_valid_obligacion": castigo_df['OBLIGACION'].isin(valid_obligacion_ids).sum(),

}

{'count_castigo_obligacion': 1170,
 'count_castigo_obligacion_in_cartera': 1130,
 'count_castigo_obligacion_in_colocacion': 1004,
 'count_castigo_valid_obligacion': 963}

In [17]:
castigo_df = (
    castigo_df
    .query('OBLIGACION.isin(@valid_obligacion_ids)')
)
castigo_df.shape

(963, 15)

CASTIGO records with known OBLIGACION are dated since 2020-04 only:

In [18]:
castigo_df['FECHA_CASTIGO'].min()

Timestamp('2020-04-01 00:00:00')

## Filtering: COLOCACION

In [19]:
colocacion_df.shape

(42009, 33)

In [20]:
colocacion_df = (
    colocacion_df
    .query('OBLIGACION.isin(@valid_obligacion_ids)')
)
colocacion_df.shape

(38641, 33)

## Filtering: CONTACTO

Question: why most of clientes have not taken any credit?

In [21]:
contacto_df.shape

(60043, 24)

In [22]:
contacto_df = (
    contacto_df
    .query('CLIENTE.isin(@valid_cliente_ids)')
)
contacto_df.shape

(22965, 24)

## Filtering: NEGOCIO

In [23]:
negocio_df.shape

(37375, 147)

In [24]:
negocio_df = (
    negocio_df
    .query('CLIENTE.isin(@valid_cliente_ids)')
)
negocio_df.shape

(22965, 147)

## Composing: CLIENTES

CLIENTES dataset is proposed to be composed from information related to clients from all provided datasets: CARTERA, COLOCACION, CONTACTO and NEGOCIO; thus, replacing CONTACTO and NEGOCIO.

Merging COLOCACION:

In [25]:
df = (
    cliente_obligacion_df
    .merge(colocacion_df[['OBLIGACION', *project_columns.COLOCACION_CLIENT_COLUMNS]], on='OBLIGACION')
    .drop(columns='OBLIGACION')
    .sort_values(by='CLIENTE')
    .reset_index(drop=True)
)

df

,CLIENTE,EDAD,ESTADO_CIVIL,ESTRATO,FECHA_NACIMIENTO,GENERO,NIVEL_ESTUDIOS,PROFESION,TIPO_UBICACION,TIPO_VIVIENDA
0,FA1,67,Casado,1.0,1955-03-30,Masculino,Primaria,DESCONOCIDA,RURAL,PROPIA
1,FA10,56,Casado,1.0,1966-06-11,Masculino,Primaria,DESCONOCIDA,RURAL,PROPIA
2,FA100,30,Soltero,1.0,1991-11-11,Masculino,Secundaria,AGRICULTURA,RURAL,FAMILIAR
3,FA1000,40,Union libre,2.0,1981-10-21,Femenino,Secundaria,SIN PROFESION,URBANA,ARRIENDO
4,FA1000,40,Union libre,2.0,1981-10-21,Femenino,Secundaria,SIN PROFESION,URBANA,ARRIENDO
...,...,...,...,...,...,...,...,...,...,...
38636,FA9997,62,Casado,2.0,1960-01-13,Masculino,Secundaria,DESCONOCIDA,URBANA,PROPIA
38637,FA9998,63,Casado,3.0,1958-10-24,Femenino,Secundaria,DESCONOCIDA,URBANA,PROPIA
38638,FA9998,63,Casado,3.0,1958-10-24,Femenino,Secundaria,DESCONOCIDA,URBANA,PROPIA
38639,FA9998,63,Casado,3.0,1958-10-24,Femenino,Secundaria,DESCONOCIDA,URBANA,PROPIA


As one client may have multiple credits (i.e. one CLIENTE with multiple OBLIGACION), there is duplicated information in COLOCACION that should be dropped:

In [26]:
any_duplicate = df['CLIENTE'].duplicated().any()

df['CLIENTE'].shape[0], any_duplicate

(38641, True)

In [27]:
df = df.drop_duplicates()
any_duplicate = df['CLIENTE'].duplicated().any()

assert any_duplicate == False, 'There are multiple rows for CLIENTE'

df['CLIENTE'].shape, any_duplicate

((22965,), False)

Merging CONTACTO:

In [28]:
df = (
    df
    .merge(contacto_df, how='left', on='CLIENTE', suffixes=('_COLOCACION', '_CONTACTO'))
    # .set_index('CLIENTE')
)

df.shape, df.columns.sort_values()

((22965, 33),
 Index(['ACTIVIDAD_CIIU_PRIMARIA', 'ACTIVIDAD_ECONOMICA', 'CLIENTE', 'EDAD',
        'ESTADO_CIVIL_COD', 'ESTADO_CIVIL_COLOCACION', 'ESTADO_CIVIL_CONTACTO',
        'ESTRATO', 'FECHA_NACIMIENTO_COLOCACION', 'FECHA_NACIMIENTO_CONTACTO',
        'GENERO_COD', 'GENERO_COLOCACION', 'GENERO_CONTACTO', 'MUJER_CABEZA',
        'NIVEL_ESTUDIOS_COD', 'NIVEL_ESTUDIOS_COLOCACION',
        'NIVEL_ESTUDIOS_CONTACTO', 'OCUPACION', 'OFICIO', 'PROFESION_COD',
        'PROFESION_COLOCACION', 'PROFESION_CONTACTO', 'RESPONSABLE_DE_HOGAR',
        'SUCURSAL', 'SUELDO_BASICO', 'TIPO_DE_CLIENTE',
        'TIPO_DE_IDENTIFICACION', 'TIPO_UBICACION_COD',
        'TIPO_UBICACION_COLOCACION', 'TIPO_UBICACION_CONTACTO',
        'TIPO_VIVIENDA_COD', 'TIPO_VIVIENDA_COLOCACION',
        'TIPO_VIVIENDA_CONTACTO'],
       dtype='object'))

Merging NEGOCIO:

In [29]:
cols = project_columns.NEGOCIO_CLIENT_COLUMNS

assert df.columns.isin(cols).any() == False, "There are duplicated columns"

df = (
    df
    .merge(negocio_df[['CLIENTE', *cols]], how='left', on='CLIENTE')
)

df.shape, df.columns.sort_values()

((22965, 34),
 Index(['ACTIVIDAD', 'ACTIVIDAD_CIIU_PRIMARIA', 'ACTIVIDAD_ECONOMICA',
        'CLIENTE', 'EDAD', 'ESTADO_CIVIL_COD', 'ESTADO_CIVIL_COLOCACION',
        'ESTADO_CIVIL_CONTACTO', 'ESTRATO', 'FECHA_NACIMIENTO_COLOCACION',
        'FECHA_NACIMIENTO_CONTACTO', 'GENERO_COD', 'GENERO_COLOCACION',
        'GENERO_CONTACTO', 'MUJER_CABEZA', 'NIVEL_ESTUDIOS_COD',
        'NIVEL_ESTUDIOS_COLOCACION', 'NIVEL_ESTUDIOS_CONTACTO', 'OCUPACION',
        'OFICIO', 'PROFESION_COD', 'PROFESION_COLOCACION', 'PROFESION_CONTACTO',
        'RESPONSABLE_DE_HOGAR', 'SUCURSAL', 'SUELDO_BASICO', 'TIPO_DE_CLIENTE',
        'TIPO_DE_IDENTIFICACION', 'TIPO_UBICACION_COD',
        'TIPO_UBICACION_COLOCACION', 'TIPO_UBICACION_CONTACTO',
        'TIPO_VIVIENDA_COD', 'TIPO_VIVIENDA_COLOCACION',
        'TIPO_VIVIENDA_CONTACTO'],
       dtype='object'))

### Comparing COLOCATION + CONTACTO

The bulk of records for each common variable are equal between both COLOCACION and CONTACTO, except for PROFESION variable:

In [30]:
indices_dict = {}

common_cols = colocacion_df.columns[colocacion_df.columns.isin(contacto_df.columns)]
common_cols.sort_values()

for col in common_cols:
    indices_dict[col] = cleaning.compare_series(df, f'{col}_COLOCACION',  f'{col}_CONTACTO')
    print(f'\n{col}')
    print(count_indices(**indices_dict[col]))


TIPO_UBICACION
{'total_rows': 22965, 'count_eq': 22876, 'count_diff': 89, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

FECHA_NACIMIENTO
{'total_rows': 22965, 'count_eq': 22946, 'count_diff': 19, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

PROFESION
{'total_rows': 22965, 'count_eq': 14766, 'count_diff': 158, 'count_na_any': 8041, 'count_na_single': 8041, 'count_na_both': 0}

GENERO
{'total_rows': 22965, 'count_eq': 22946, 'count_diff': 19, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

NIVEL_ESTUDIOS
{'total_rows': 22965, 'count_eq': 22703, 'count_diff': 251, 'count_na_any': 11, 'count_na_single': 5, 'count_na_both': 6}

ESTADO_CIVIL
{'total_rows': 22965, 'count_eq': 22505, 'count_diff': 377, 'count_na_any': 83, 'count_na_single': 38, 'count_na_both': 45}

TIPO_VIVIENDA
{'total_rows': 22965, 'count_eq': 22183, 'count_diff': 673, 'count_na_any': 109, 'count_na_single': 109, 'count_na_both': 0}


#### PROFESION

PROFESION column is present in both COLOCACION and CONTACTO datasets having three different types of value to identify missing values:
- NA value in CONTACTO dataset only.
- Category "DESCONOCIDA" in both COLOCACION and CONTACTO datasets.
- Category "SIN PROFESION" in COLOCACION dataset only. All "SIN PROFESION" values correspond to NA values in PROFESION_CONTACTO.

In [31]:
profesion_df = df[['PROFESION_COLOCACION', 'PROFESION_CONTACTO']]

index_na = profesion_df.isna()
index_sin_profesion = profesion_df == 'SIN PROFESION'

count_na = index_na.sum()
count_sin_profesion = index_sin_profesion.sum()

display({
    "count_colocacion_na": count_na['PROFESION_COLOCACION'],
    "count_contacto_na": count_na['PROFESION_CONTACTO'],
    "count_colocacion_sin_profesion": count_sin_profesion['PROFESION_COLOCACION'],
    "count_contacto_sin_profesion": count_sin_profesion['PROFESION_CONTACTO'],
    "all_colocacion_sin_profesion_equal_contacto_na": (
        profesion_df
        .loc[index_sin_profesion['PROFESION_COLOCACION'], 'PROFESION_CONTACTO']
        .isna()
        .all()
    ),
})

profesion_df


{'count_colocacion_na': 0,
 'count_contacto_na': 8041,
 'count_colocacion_sin_profesion': 7830,
 'count_contacto_sin_profesion': 0,
 'all_colocacion_sin_profesion_equal_contacto_na': True}

,PROFESION_COLOCACION,PROFESION_CONTACTO
0,DESCONOCIDA,DESCONOCIDA
1,DESCONOCIDA,DESCONOCIDA
2,AGRICULTURA,AGRICULTURA
3,SIN PROFESION,NaN
4,DESCONOCIDA,DESCONOCIDA
...,...,...
22960,DESCONOCIDA,DESCONOCIDA
22961,SIN PROFESION,NaN
22962,DESCONOCIDA,DESCONOCIDA
22963,DESCONOCIDA,DESCONOCIDA


PROFESION values equal to "DESCONOCIDA" have following considerations:
- There are 49 records in COLOCACION with actual values that were exported as "DESCONOCIDA" in CONTACTO.
- There are 58 records exported as "DESCONOCIDA" in COLOCACION that have actual values in CONTACTO.
- There are 103 records exported as "DESCONOCIDA" in COLOCACION that are NA values in CONTACTO.

In [32]:
index_desconocida = profesion_df[['PROFESION_COLOCACION', 'PROFESION_CONTACTO']] == 'DESCONOCIDA'
index_1 = (
    ~index_desconocida['PROFESION_COLOCACION']
    & index_desconocida['PROFESION_CONTACTO']
)
index_2 = (
    index_desconocida['PROFESION_COLOCACION']
    & ~index_desconocida['PROFESION_CONTACTO']
    & ~(index_na['PROFESION_CONTACTO'])
)
index_3 = (
    index_desconocida['PROFESION_COLOCACION']
    & index_na['PROFESION_CONTACTO']
)

display({
    'count_COLOCACION_value_CONTACTO_eq_DESCONOCIDA': index_1.sum(),
    'count_COLOCACION_eq_DESCONOCIDA_CONTACTO_value': index_2.sum(),
    'count_COLOCACION_eq_DESCONOCIDA_CONTACTO_na': index_3.sum(),
})

profesion_df[index_1 | index_2]

{'count_COLOCACION_value_CONTACTO_eq_DESCONOCIDA': 45,
 'count_COLOCACION_eq_DESCONOCIDA_CONTACTO_value': 51,
 'count_COLOCACION_eq_DESCONOCIDA_CONTACTO_na': 98}

,PROFESION_COLOCACION,PROFESION_CONTACTO
491,DESCONOCIDA,ADMON DE EMPRESAS
510,DESCONOCIDA,INGENIERIA INDUSTRIAL
525,DESCONOCIDA,ADMON AGROPECUARIA
597,DESCONOCIDA,AGRICULTURA
611,ECONOMIA,DESCONOCIDA
...,...,...
20211,ADMON DE EMPRESAS,DESCONOCIDA
20219,AGRICULTURA,DESCONOCIDA
20299,AGRICULTURA,DESCONOCIDA
20366,AGRICULTURA,DESCONOCIDA


Composing most complete PROFESION data defining NaN values as unique value for missing values:

In [33]:
profesion_series = profesion_df['PROFESION_COLOCACION'].copy()
profesion_series[index_2] = profesion_df.loc[index_2, 'PROFESION_CONTACTO']
profesion_series = profesion_series.replace({'DESCONOCIDA': np.nan, 'SIN PROFESION': np.nan})
profesion_series

0                NaN
1                NaN
2        AGRICULTURA
3                NaN
4                NaN
            ...     
22960            NaN
22961            NaN
22962            NaN
22963            NaN
22964            NaN
Name: PROFESION_COLOCACION, Length: 22965, dtype: object

### Composing CLIENTES

Compose demographics considering variable values that are equal in both COLOCACION and CONTACTO (i.e. the bulk of data), setting different values as NaN to discard inconsistencies, except for PROFESION which is explicitly set using composed PROFESION series:

In [34]:
for col in common_cols[common_cols != 'PROFESION']:
    df[col] = df[f'{col}_COLOCACION']

    ind_eq = indices_dict[col]['eq']
    df.loc[~ind_eq, col] = np.nan

df['PROFESION'] = profesion_series

Dropping comparison columns:

In [35]:
re_columns_pattern = fr"({'|'.join(common_cols)})_(COLOCACION|CONTACTO)"
re_columns_pattern

'(TIPO_UBICACION|FECHA_NACIMIENTO|PROFESION|GENERO|NIVEL_ESTUDIOS|ESTADO_CIVIL|TIPO_VIVIENDA)_(COLOCACION|CONTACTO)'

In [36]:
ind = df.columns.str.match(re_columns_pattern)
dropping_cols = df.columns[ind]
dropping_cols.sort_values()

Index(['ESTADO_CIVIL_COLOCACION', 'ESTADO_CIVIL_CONTACTO',
       'FECHA_NACIMIENTO_COLOCACION', 'FECHA_NACIMIENTO_CONTACTO',
       'GENERO_COLOCACION', 'GENERO_CONTACTO', 'NIVEL_ESTUDIOS_COLOCACION',
       'NIVEL_ESTUDIOS_CONTACTO', 'PROFESION_COLOCACION', 'PROFESION_CONTACTO',
       'TIPO_UBICACION_COLOCACION', 'TIPO_UBICACION_CONTACTO',
       'TIPO_VIVIENDA_COLOCACION', 'TIPO_VIVIENDA_CONTACTO'],
      dtype='object')

In [37]:
clientes_df = df.drop(columns=dropping_cols)

clientes_df.columns.sort_values()

Index(['ACTIVIDAD', 'ACTIVIDAD_CIIU_PRIMARIA', 'ACTIVIDAD_ECONOMICA',
       'CLIENTE', 'EDAD', 'ESTADO_CIVIL', 'ESTADO_CIVIL_COD', 'ESTRATO',
       'FECHA_NACIMIENTO', 'GENERO', 'GENERO_COD', 'MUJER_CABEZA',
       'NIVEL_ESTUDIOS', 'NIVEL_ESTUDIOS_COD', 'OCUPACION', 'OFICIO',
       'PROFESION', 'PROFESION_COD', 'RESPONSABLE_DE_HOGAR', 'SUCURSAL',
       'SUELDO_BASICO', 'TIPO_DE_CLIENTE', 'TIPO_DE_IDENTIFICACION',
       'TIPO_UBICACION', 'TIPO_UBICACION_COD', 'TIPO_VIVIENDA',
       'TIPO_VIVIENDA_COD'],
      dtype='object')

Validating that values for EDAD are based on FECHA_NACIMIENTO when exporting data on 2022-03:

In [38]:
clientes_df[['EDAD', 'FECHA_NACIMIENTO']].isna().sum()

EDAD                 0
FECHA_NACIMIENTO    19
dtype: int64

In [39]:
(clientes_df['EDAD']  + clientes_df['FECHA_NACIMIENTO'].dt.year).value_counts(dropna=False)

2021.0    12770
2022.0    10176
NaN          19
dtype: int64

## Updating CARTERA + COLOCACION

Updating CARTERA by dropping columns that were assigned to CLIENTES:

In [40]:
dropping_cols = cartera_df.columns[cartera_df.columns.isin(clientes_df.columns)]
dropping_cols = dropping_cols[dropping_cols != 'CLIENTE']

dropping_cols.sort_values()

Index(['TIPO_UBICACION'], dtype='object')

In [41]:
cartera_df = cartera_df.drop(columns=dropping_cols)

cartera_df.columns.sort_values()

Index(['CALIFICACION_CIERRE', 'CAPITAL_VEN', 'CLIENTE', 'COD_LINEA',
       'COD_MODALIDAD', 'COMISION', 'CUOTAS_PACTADAS', 'CUOTAS_PENDIENTES',
       'DIAS_VENCIDO', 'FECHA_APROBA', 'FECHA_CIERRE', 'FECHA_DESEMBOLSO',
       'FECHA_PROXIMO_PAGO', 'FECHA_SOLICITUD', 'FECHA_ULT_PAGO',
       'FECHA_VENCIMIENTO_FINAL', 'INTERES_VEN', 'LINEA', 'MODALIDAD', 'MONTO',
       'MORA', 'MUNICIPIO_CLIENTE', 'MUNICIPIO_LAT', 'MUNICIPIO_LON',
       'OBLIGACION', 'OTROS', 'PERIODICIDAD_PAGO', 'PORCENTAJE_PAGO', 'REGION',
       'SALDO', 'SEGURO_VIDA', 'SUCURSAL_COD', 'SUCURSAL_REAL', 'TASA_ANUAL',
       'TASA_PERIODICA', 'TIPO_CLIENTE', 'TIPO_CLIENTE_COD', 'TIPO_CREDITO',
       'VALOR_CUOTA', 'VENCIDA'],
      dtype='object')

Updating COLOCACION dataset by dropping columns that were assigned to CLIENTES dataset, adding CLIENTE column, and adding DEFAULT column:

In [42]:
dropping_cols = colocacion_df.columns[colocacion_df.columns.isin(clientes_df.columns)]

dropping_cols.sort_values()

Index(['EDAD', 'ESTADO_CIVIL', 'ESTRATO', 'FECHA_NACIMIENTO', 'GENERO',
       'NIVEL_ESTUDIOS', 'PROFESION', 'TIPO_UBICACION', 'TIPO_VIVIENDA'],
      dtype='object')

In [43]:
castigo_df = (
    castigo_df[['OBLIGACION']]
    .assign(DEFAULT=True)
)

colocacion_df = (
    colocacion_df
    .drop(columns=dropping_cols)
    .merge(cliente_obligacion_df, on='OBLIGACION')
    .merge(castigo_df, on='OBLIGACION', how='left')
)


colocacion_df.shape, colocacion_df.columns.sort_values()

((38641, 26),
 Index(['ANO_CONTABILIZA', 'CLIENTE', 'CODEUDOR', 'COD_DESTINACION',
        'COD_LINEA', 'COD_MODALIDAD', 'CREDITOS_VIGENTES', 'DEFAULT',
        'DESTINACION', 'DIAS_CICLO_CREDITO', 'FECHA_APROBA', 'FECHA_DESEMBOLSO',
        'FECHA_PAGO', 'FECHA_SOLICITUD', 'LINEA', 'MODALIDAD', 'MONTO',
        'NRO_CUOTAS', 'OBLIGACION', 'OBSERVACIONES', 'PERIODICIDAD_PAGO',
        'TASA_ANUAL', 'TIPO_CREDITO', 'VALOR_CUOTA', 'VALOR_DESEMBOLSADO',
        'VALOR_REFINANCIADO'],
       dtype='object'))

In [44]:
colocacion_df['DEFAULT'].value_counts(dropna=False)

NaN     37678
True      963
Name: DEFAULT, dtype: int64

## Comparing CARTERA + COLOCACION

Common columns between CARTERA and COLOCACION:

In [45]:
common_cols = pd.Series(project_columns.CARTERA_COLOCACION_COLUMNS)

common_cols

0           COD_LINEA
1       COD_MODALIDAD
2        FECHA_APROBA
3    FECHA_DESEMBOLSO
4     FECHA_SOLICITUD
5               LINEA
6               MONTO
7          TASA_ANUAL
dtype: object

Filtering CARTERA:

In [46]:
columns = ['CLIENTE', 'OBLIGACION', *common_cols]
df = (
    cartera_df
    [['FECHA_CIERRE', *columns]]
    .drop_duplicates(subset=columns)
    .sort_values(by='OBLIGACION')
    .reset_index(drop=True)
)

df.columns.sort_values()

Index(['CLIENTE', 'COD_LINEA', 'COD_MODALIDAD', 'FECHA_APROBA', 'FECHA_CIERRE',
       'FECHA_DESEMBOLSO', 'FECHA_SOLICITUD', 'LINEA', 'MONTO', 'OBLIGACION',
       'TASA_ANUAL'],
      dtype='object')

Validating there are no duplicated information within CARTERA for each OBLIGACION redarding common columns:

In [47]:
any_duplicated = df['OBLIGACION'].duplicated().any()

assert any_duplicate == False, 'There are multiple data for CLIENTE'

Merging COLOCACION:

In [48]:
df = (
    df
    .merge(colocacion_df[['OBLIGACION', *common_cols]], on='OBLIGACION', suffixes=('_CARTERA', '_COLOCACION'))
    .set_index('OBLIGACION')
)

df.columns.sort_values()

Index(['CLIENTE', 'COD_LINEA_CARTERA', 'COD_LINEA_COLOCACION',
       'COD_MODALIDAD_CARTERA', 'COD_MODALIDAD_COLOCACION',
       'FECHA_APROBA_CARTERA', 'FECHA_APROBA_COLOCACION', 'FECHA_CIERRE',
       'FECHA_DESEMBOLSO_CARTERA', 'FECHA_DESEMBOLSO_COLOCACION',
       'FECHA_SOLICITUD_CARTERA', 'FECHA_SOLICITUD_COLOCACION',
       'LINEA_CARTERA', 'LINEA_COLOCACION', 'MONTO_CARTERA',
       'MONTO_COLOCACION', 'TASA_ANUAL_CARTERA', 'TASA_ANUAL_COLOCACION'],
      dtype='object')

Most values from common columns between CARTERA and COLOCACION are consistently equal but except for FECHA_DESEMBOLSO:

In [49]:
indices_dict = {}

for col in common_cols:
    indices_dict[col] = cleaning.compare_series(df, f'{col}_CARTERA',  f'{col}_COLOCACION')
    print(f'\n{col}')
    print(count_indices(**indices_dict[col]))


COD_LINEA
{'total_rows': 38641, 'count_eq': 38641, 'count_diff': 0, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

COD_MODALIDAD
{'total_rows': 38641, 'count_eq': 38641, 'count_diff': 0, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

FECHA_APROBA
{'total_rows': 38641, 'count_eq': 38641, 'count_diff': 0, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

FECHA_DESEMBOLSO
{'total_rows': 38641, 'count_eq': 38484, 'count_diff': 157, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

FECHA_SOLICITUD
{'total_rows': 38641, 'count_eq': 38641, 'count_diff': 0, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

LINEA
{'total_rows': 38641, 'count_eq': 38641, 'count_diff': 0, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

MONTO
{'total_rows': 38641, 'count_eq': 38641, 'count_diff': 0, 'count_na_any': 0, 'count_na_single': 0, 'count_na_both': 0}

TASA_ANUAL
{'total_rows': 38641, 'count_eq': 38641, 'count_diff': 0, 'count

### FECHA_DESEMBOLSO

In [50]:
ind = indices_dict['FECHA_DESEMBOLSO']['diff']
d = df.loc[ind, ['CLIENTE', 'FECHA_DESEMBOLSO_CARTERA', 'FECHA_DESEMBOLSO_COLOCACION', 'FECHA_CIERRE']]

Besides the differences of FECHA_DESEMBOLSO between CARTERA and COLOCACION, all values are consistently part of month corresponding to FECHA_CIERRE:

In [51]:
{
    "count_desembolso_diff": d.shape[0],
    "count_cartera_greater_colocacion": (d['FECHA_DESEMBOLSO_COLOCACION'] > d['FECHA_DESEMBOLSO_CARTERA']).sum(),
    "count_cartera_lower_colocacion": (d['FECHA_DESEMBOLSO_COLOCACION'] < d['FECHA_DESEMBOLSO_CARTERA']).sum(),
    "consistent_desembolso_cartera": (d['FECHA_DESEMBOLSO_CARTERA'] <= d['FECHA_CIERRE']).all(),
    "consistent_desembolso_colocacion": (d['FECHA_DESEMBOLSO_COLOCACION'] <= d['FECHA_CIERRE']).all()
}

{'count_desembolso_diff': 157,
 'count_cartera_greater_colocacion': 114,
 'count_cartera_lower_colocacion': 43,
 'consistent_desembolso_cartera': True,
 'consistent_desembolso_colocacion': True}

Differences range from 1 day up until 11 days:

In [52]:
abs_dif = (d['FECHA_DESEMBOLSO_COLOCACION'] - d['FECHA_DESEMBOLSO_CARTERA']).abs()
abs_dif.describe()

count                          157
mean     1 days 16:30:34.394904458
std      1 days 09:15:56.719587379
min                1 days 00:00:00
25%                1 days 00:00:00
50%                1 days 00:00:00
75%                2 days 00:00:00
max               11 days 00:00:00
dtype: object

In [53]:
pd.concat([d, abs_dif.rename('ABS_DIFF')], axis=1).sort_values(by='ABS_DIFF')

,CLIENTE,FECHA_DESEMBOLSO_CARTERA,FECHA_DESEMBOLSO_COLOCACION,FECHA_CIERRE,ABS_DIFF
OBLIGACION,,,,,
191004186,FA23262,2020-03-02,2020-03-03,2020-03-31,1 days
193002462,FA8075,2020-02-12,2020-02-13,2020-02-29,1 days
193002005,FA18736,2019-08-02,2019-08-01,2019-08-31,1 days
192003765,FA4705,2020-10-08,2020-10-09,2020-10-31,1 days
192003587,FA10579,2020-07-31,2020-07-30,2020-07-31,1 days
...,...,...,...,...,...
179000106,FA6036,2017-04-07,2017-04-12,2017-04-30,5 days
175000095,FA5799,2017-05-03,2017-05-08,2017-05-31,5 days
1913000361,FA7541,2020-02-11,2020-02-17,2020-02-29,6 days


As CARTERA is the dataset with most errors, FECHA_DESEMBOLSO values from COLOCACION dataset are considered as true values.

### Updating CARTERA

In [54]:
cartera_df = cartera_df.drop(columns=common_cols)

cartera_df.columns.sort_values()

Index(['CALIFICACION_CIERRE', 'CAPITAL_VEN', 'CLIENTE', 'COMISION',
       'CUOTAS_PACTADAS', 'CUOTAS_PENDIENTES', 'DIAS_VENCIDO', 'FECHA_CIERRE',
       'FECHA_PROXIMO_PAGO', 'FECHA_ULT_PAGO', 'FECHA_VENCIMIENTO_FINAL',
       'INTERES_VEN', 'MODALIDAD', 'MORA', 'MUNICIPIO_CLIENTE',
       'MUNICIPIO_LAT', 'MUNICIPIO_LON', 'OBLIGACION', 'OTROS',
       'PERIODICIDAD_PAGO', 'PORCENTAJE_PAGO', 'REGION', 'SALDO',
       'SEGURO_VIDA', 'SUCURSAL_COD', 'SUCURSAL_REAL', 'TASA_PERIODICA',
       'TIPO_CLIENTE', 'TIPO_CLIENTE_COD', 'TIPO_CREDITO', 'VALOR_CUOTA',
       'VENCIDA'],
      dtype='object')

## Validating joining function

In [55]:
(car, cli, col) = datasets.read_joining_datasets(dir_path=RAW_DATA_PATH)

In [56]:
comparison = (   
    cleaning.compare_dataframes(cartera_df, car, sort_by=['CLIENTE', 'OBLIGACION', 'FECHA_CIERRE']),
    cleaning.compare_dataframes(clientes_df, cli, sort_by=['CLIENTE']),
    cleaning.compare_dataframes(colocacion_df, col, sort_by=['CLIENTE', 'OBLIGACION']),
)

assert comparison == (True, True, True), "Dataframes are different"